In [1]:
import pandas as pd

_pipeline_module_file = '/home/mlops/airflow/dags/test_pipeline.py'


In [44]:
%%writefile {_pipeline_module_file}
from delta import configure_spark_with_delta_pip
import pyspark
import datetime
import logging
import os

from spotipy import SpotifyClientCredentials
import spotipy
from airflow import DAG
from airflow.decorators import task, task_group
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark.pandas as ps
import pandas as pd

from collections import Counter
import re


log = logging.getLogger(__name__)
spark = None
db = None





with DAG(
    dag_id="testDAG",
    description='test_dag',
    schedule_interval=None,
    start_date= datetime.datetime(year=2022, month=2, day=1)
) as dag:

    if spark is None:
    #Initialize Spark
        builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

        spark = configure_spark_with_delta_pip(builder).getOrCreate()

    @task(task_id='SILBER_ticketmaster_augmented_table_start')
    def ingest_spotify_data_start():
        #read deltalake table
        ticketmaster = spark.read.format('delta').load('file:///home/mlops/project/DeltaLake/bronze_data/ticketmaster_table')

        #read spotify data
        spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='7b4d7d2626b04188a79a02ecdda845e2', client_secret='e01be9dfdd854f74bfe0d0af10c5a4ee'))

        print(f"SPOTIPY: {spotify}")
        return ticketmaster.to_pandas_on_spark().to_dict()

    @task(task_id='test')
    def test(some):
        print("SDFZUIOPOIUGF:", some)
        print(pd.DataFrame(some).head())
        return some
    @task_group(group_id='group_1')
    def group_1(ticketmaster_dict):

        # The @tasks below can be defined outside function `group_1`
        # What matters is where they are referenced
        # for index, tick in ticketmaster_df.iterrows():
        tasks = list()
        spotify_list = list()
        print('TYPE:', type(ticketmaster_dict))
        # for i in range(len(ticketmaster_dict['name'])):
        for i in range(5):
            @task(task_id=f"task_1_{i}")
            def task_1(spotify_list, ticketmaster_dict, index):
                # l.append(tick)
                # l += tick['Age'][0]
                print(ticketmaster_dict['name'][index])
                return {
                    'spotify_list': spotify_list,
                    'ticketmaster_dict': ticketmaster_dict
                }
            # tasks are referenced here
            # task_1_result = task_1(l, tick)
            tasks.append(task_1(spotify_list, ticketmaster_dict, i))


        # @task
        # def task_1(l_tick):
        #         l, ticketmaster_df = l_tick
        #
        #         print(l, ticketmaster_df)
        #         return l
        # task_1_result = task_1(l_tick)
        # sending this list to `group_2`
        return tasks


    # group_1(ingest_spotify_data_start())
    # test(ingest_spotify_data_start())\

    start = ingest_spotify_data_start()
    do = group_1(start)


Overwriting /home/mlops/airflow/dags/test_pipeline.py


In [32]:
from delta import configure_spark_with_delta_pip
import pyspark

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [5]:
ticketmaster = spark.read.format('delta').load('file:///home/mlops/project/DeltaLake/bronze_data/ticketmaster_table')


In [7]:


bitches = ticketmaster.to_pandas_on_spark().to_dict()



22/02/05 13:32:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/05 13:32:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/05 13:32:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


name
id
locale
distance
units
dates
classifications
_links
price_min
price_max
_embedded


In [21]:
import pyspark.pandas as ps

for x in range(len(bitches)):
    print(x)




0
1
2
3
4
5
6
7
8
9
10


In [29]:
bitches['name'][807]


'BLVTH'